In [1]:
import numpy as np 
import pandas as pd 
import os
import gc
from joblib import load, dump

In [2]:
# Paths
BASE_DIR = 'C:/Users/agust/Documents/MaestriaCD/LAB3/final/'
#PATH_TO_PREDICT = os.path.join(BASE_DIR, "Dataset/predecir.parquet")
#PATH_TO_PREDICT = os.path.join(BASE_DIR, "Dataset/predecir_producto.parquet")
PATH_TO_PREDICT = os.path.join(BASE_DIR,"predecir_producto_autogluon.parquet")
#PATH_TO_ESCALA = os.path.join(BASE_DIR, "Dataset/factores_escala_por_producto.csv")
#PATH_TO_PREDICT = os.path.join(BASE_DIR, "Dataset/predecir_producto_201910.parquet")
PATH_TO_MODELS = os.path.join(BASE_DIR, "LGBM/models")
#PATH_TO_MODELS_BAJO_VOL = os.path.join(BASE_DIR, "LGBM/models/bajos")
PATH_TO_TEMP_FILES = os.path.join(BASE_DIR, "LGBM/optuna/optuna_temp_artifacts")
PATH_TO_OPTUNA_ARTIFACTS = os.path.join(BASE_DIR, "LGBM/optuna/optuna_artifacts")

SEED = 42

In [4]:
# Datos Tabulares
dataset = pd.read_parquet(PATH_TO_PREDICT)
productos_a_predecir= pd.read_csv(os.path.join(BASE_DIR, "product_predict.txt"), sep='\t')

In [5]:
dataset = dataset.loc[dataset['periodo'] == 201912]

In [6]:
dataset.head()

,periodo,product_id,tn,avg_tn,std_tn,clientes_distintos,plan_precios_cuidados,cust_request_qty,cust_request_tn,inicio_vida_p,...,otros_avg_lag10,otros_avg_lag11,otros_avg_lag12,otros_avg_lag13,tn_pred_auto,n_meses_hist,tn_pred_auto_delta_a_tn,tn_pred_auto_delta_a_clase_absoluto,tn_pred_auto_delta_a_clase,tn_pred_auto_delta_a_clase_ratio
1,201912,20001,1505.0,2.519531,15.929688,597,0,407,1650.0,201701,...,95.2500,95.1250,92.7500,127.8750,NaN,NaN,NaN,NaN,NaN,NaN
3,201912,20002,1087.0,1.821289,17.156250,597,0,217,1092.0,201701,...,101.0625,95.3125,106.0625,129.1250,NaN,NaN,NaN,NaN,NaN,NaN
5,201912,20003,892.5,1.495117,8.257812,597,0,314,970.0,201701,...,65.5625,66.8125,71.5625,94.5000,NaN,NaN,NaN,NaN,NaN,NaN
7,201912,20004,638.0,1.068359,4.445312,597,0,313,678.5,201701,...,75.1250,80.5625,77.0000,106.4375,NaN,NaN,NaN,NaN,NaN,NaN
9,201912,20005,593.0,0.993652,5.593750,597,0,269,630.5,201701,...,76.1250,85.0000,83.3125,116.1875,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
# renombrar la columna tn por producto_total_tn
dataset.rename(columns={'tn': 'producto_total_tn'}, inplace=True)

In [8]:
dataset.head()

,periodo,product_id,producto_total_tn,avg_tn,std_tn,clientes_distintos,plan_precios_cuidados,cust_request_qty,cust_request_tn,inicio_vida_p,...,otros_avg_lag10,otros_avg_lag11,otros_avg_lag12,otros_avg_lag13,tn_pred_auto,n_meses_hist,tn_pred_auto_delta_a_tn,tn_pred_auto_delta_a_clase_absoluto,tn_pred_auto_delta_a_clase,tn_pred_auto_delta_a_clase_ratio
1,201912,20001,1505.0,2.519531,15.929688,597,0,407,1650.0,201701,...,95.2500,95.1250,92.7500,127.8750,NaN,NaN,NaN,NaN,NaN,NaN
3,201912,20002,1087.0,1.821289,17.156250,597,0,217,1092.0,201701,...,101.0625,95.3125,106.0625,129.1250,NaN,NaN,NaN,NaN,NaN,NaN
5,201912,20003,892.5,1.495117,8.257812,597,0,314,970.0,201701,...,65.5625,66.8125,71.5625,94.5000,NaN,NaN,NaN,NaN,NaN,NaN
7,201912,20004,638.0,1.068359,4.445312,597,0,313,678.5,201701,...,75.1250,80.5625,77.0000,106.4375,NaN,NaN,NaN,NaN,NaN,NaN
9,201912,20005,593.0,0.993652,5.593750,597,0,269,630.5,201701,...,76.1250,85.0000,83.3125,116.1875,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
#SOLO SI HAY PESOS
# Eliminar pesos 
#dataset['ultima_tn']

In [9]:
dataset.shape

(927, 285)

In [10]:
#unir en un solo dataset
dataset = (
    dataset
    .merge(productos_a_predecir, on='product_id', how='inner')
)

In [65]:
#del dataset['volumen_alto']

In [11]:
dataset.shape

(780, 285)

In [12]:
def tipos_correctos(df, verbose=True):
    start_mem = df.memory_usage(deep=True).sum() / 1024**2

    for col in df.columns:
        col_type = df[col].dtype

        #   - Datetime
        # 1) Datetime → int64 (ns) → float32
        if pd.api.types.is_datetime64_any_dtype(col_type):
            # view() extrae los nanosegundos desde epoch
            df[col] = df[col].view('int64').astype('float32')
            continue


        # Solo nos ocupamos de numéricos
        if not pd.api.types.is_numeric_dtype(col_type):
            if not pd.api.types.is_categorical_dtype(col_type):
                df[col] = df[col].astype('category')
            continue

        c_min, c_max = df[col].min(), df[col].max()
        has_na = df[col].isnull().any()

        # --- ENTEROS ---
        if pd.api.types.is_integer_dtype(col_type):
            # 1) Sin nulos → numpy ints
            if not has_na:
                if c_min >= np.iinfo(np.int8).min  and c_max <= np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min >= np.iinfo(np.int16).min and c_max <= np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min >= np.iinfo(np.int32).min and c_max <= np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                else:
                    df[col] = df[col].astype(np.int64)


        # --- FLOTANTES ---
        else:
            df[col] = df[col].astype(np.float32)

    end_mem = df.memory_usage(deep=True).sum() / 1024**2
    if verbose:
        print(f'Uso de memoria inicial del DataFrame: {start_mem:.2f} MB')
        print(f'Uso de memoria final del DataFrame:   {end_mem:.2f} MB')
        print(f'Memoria reducida en un {(100*(start_mem-end_mem)/start_mem):.2f}%')
    return df

In [13]:
dataset = tipos_correctos(dataset)

Uso de memoria inicial del DataFrame: 0.47 MB
Uso de memoria final del DataFrame:   0.82 MB
Memoria reducida en un -75.83%


C:\Users\agust\AppData\Local\Temp\ipykernel_26012\57235330.py:17: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  if not pd.api.types.is_categorical_dtype(col_type):
C:\Users\agust\AppData\Local\Temp\ipykernel_26012\57235330.py:17: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  if not pd.api.types.is_categorical_dtype(col_type):


In [14]:
pred=dataset.copy()

In [15]:
pred.head(20)

,periodo,product_id,producto_total_tn,avg_tn,std_tn,clientes_distintos,plan_precios_cuidados,cust_request_qty,cust_request_tn,inicio_vida_p,...,otros_avg_lag10,otros_avg_lag11,otros_avg_lag12,otros_avg_lag13,tn_pred_auto,n_meses_hist,tn_pred_auto_delta_a_tn,tn_pred_auto_delta_a_clase_absoluto,tn_pred_auto_delta_a_clase,tn_pred_auto_delta_a_clase_ratio
0,201912,20001,1505.000,2.519531,15.929688,597,0,407,1650.000,201701,...,95.25000,95.12500,92.75000,127.87500,NaN,NaN,NaN,NaN,NaN,NaN
1,201912,20002,1087.000,1.821289,17.156250,597,0,217,1092.000,201701,...,101.06250,95.31250,106.06250,129.12500,NaN,NaN,NaN,NaN,NaN,NaN
2,201912,20003,892.500,1.495117,8.257812,597,0,314,970.000,201701,...,65.56250,66.81250,71.56250,94.50000,NaN,NaN,NaN,NaN,NaN,NaN
3,201912,20004,638.000,1.068359,4.445312,597,0,313,678.500,201701,...,75.12500,80.56250,77.00000,106.43750,NaN,NaN,NaN,NaN,NaN,NaN
4,201912,20005,593.000,0.993652,5.593750,597,0,269,630.500,201701,...,76.12500,85.00000,83.31250,116.18750,NaN,NaN,NaN,NaN,NaN,NaN
5,201912,20006,417.250,0.698730,4.679688,597,0,250,439.750,201701,...,90.18750,94.43750,85.68750,89.25000,NaN,NaN,NaN,NaN,NaN,NaN
6,201912,20007,390.500,0.653809,4.164062,597,0,338,398.750,201701,...,88.12500,80.18750,88.18750,121.93750,NaN,NaN,NaN,NaN,NaN,NaN
7,201912,20008,195.375,0.327148,1.662109,597,0,171,225.375,201701,...,68.43750,61.15625,52.87500,59.78125,NaN,NaN,NaN,NaN,NaN,NaN
8,201912,20009,495.000,0.829102,5.867188,597,0,297,523.000,201701,...,119.37500,117.68750,118.68750,162.87500,NaN,NaN,NaN,NaN,NaN,NaN
9,201912,20010,359.500,0.602539,4.742188,597,0,172,440.500,201701,...,131.37500,125.43750,115.00000,144.75000,NaN,NaN,NaN,NaN,NaN,NaN


# Predicción

In [16]:
class LGBMEnsemble:
    def __init__(self, models):
        self.models = models

    def predict(self, X):
        preds = np.mean([model.predict(X) for model in self.models], axis=0)
        return preds

# Cargar todos los modelos .joblib en la carpeta
ensemble_models = [
    load(os.path.join(PATH_TO_MODELS, fname))
    for fname in os.listdir(PATH_TO_MODELS)
    if fname.endswith('.joblib') and fname.startswith('lgb_model_')
]

# Crear el modelo de ensamble
ensemble_model = LGBMEnsemble(ensemble_models)

# Guardar el ensamble
path_guardado = os.path.join(PATH_TO_MODELS, "modelo_lgbm_ensemble.joblib")
dump(ensemble_model, path_guardado)
print(f"✅ Ensamble guardado en: {path_guardado}")


✅ Ensamble guardado en: C:/Users/agust/Documents/MaestriaCD/LAB3/final/LGBM/models\modelo_lgbm_ensemble.joblib


In [17]:
#predicción con ensemble

modelo = load(os.path.join(PATH_TO_MODELS, f"modelo_lgbm_ensemble.joblib"))
# Realizar predicciones
pred["clase"] = modelo.predict(pred)

In [18]:
pred.sample(5)

,periodo,product_id,producto_total_tn,avg_tn,std_tn,clientes_distintos,plan_precios_cuidados,cust_request_qty,cust_request_tn,inicio_vida_p,...,otros_avg_lag11,otros_avg_lag12,otros_avg_lag13,tn_pred_auto,n_meses_hist,tn_pred_auto_delta_a_tn,tn_pred_auto_delta_a_clase_absoluto,tn_pred_auto_delta_a_clase,tn_pred_auto_delta_a_clase_ratio,clase
762,201912,21222,0.021835,0.000037,0.000552,597,0,5,0.021835,201701,...,0.041504,0.057556,0.063354,NaN,NaN,NaN,NaN,NaN,NaN,0.139590
717,201912,21105,0.434814,0.000729,0.005558,597,0,51,0.434814,201811,...,20.312500,16.234375,20.156250,NaN,NaN,NaN,NaN,NaN,NaN,0.175129
700,201912,21057,0.675781,0.001132,0.007084,597,1,118,0.675781,201701,...,1.033203,0.872559,0.911621,NaN,NaN,NaN,NaN,NaN,NaN,-0.046814
94,201912,20107,86.062500,0.144165,0.900879,597,1,156,93.687500,201701,...,94.187500,91.437500,127.062500,NaN,NaN,NaN,NaN,NaN,NaN,-6.272527
333,201912,20414,28.515625,0.047760,0.306396,597,0,188,28.515625,201904,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.772727


# armar prediccion

In [19]:
pred["tn_predicha"]=pred["clase"]+pred["producto_total_tn"]
pred.loc[pred['tn_predicha'] < 0, 'tn_predicha'] = 0


In [20]:
pred.rename(columns={"tn_predicha": "tn"}, inplace=True)

In [21]:
pred[['product_id', 'tn']].to_csv('prediccion_lgbm_autogluon2.csv', index=False)